# PRUEBA DE RAG_PIPELINE

## 1. Usando los textos extraidos con web scraping desde las url


### 1. Cargar a Documentos de Langchain los txt de web scraping

In [1]:
from langchain.document_loaders import DirectoryLoader

loader1 = DirectoryLoader("../data/data_web_scraping", glob="**/*.txt")
docs1 = loader1.load()

### 2. Dividir Texto en Chunks

1000 tokens: Equivalente a ~1 página de texto, balance entre contexto y eficiencia

200 de overlap: Permite mantener contexto entre chunks adyacentes

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits1 = text_splitter.split_documents(docs1)

In [3]:
splits1

[Document(metadata={'source': '..\\data\\data_web_scraping\\texto_extraido_url_1_tarjetas_de_credito.txt'}, page_content='¿Qué quieres buscar hoy? Tus ofertas favoritas\n\nEncuentra la tarjeta de crédito Tuya perfecta para ti y disfruta sus beneficios Con las tarjetas de crédito Tuya podrás comprar lo que más te gusta y disfrutar descuentos exclusivos en almacenes Éxito y Carulla. Conócelas todas y solicita la tuya.\n\nConoce los beneficios de nuestras tarjetas: Disfruta descuentos exclusivos cada semana en almacenes Éxito y Carulla. Con nuestras tarjetas podrás construir un buen historial crediticio.'),
 Document(metadata={'source': '..\\data\\data_web_scraping\\texto_extraido_url_1_tarjetas_de_credito.txt'}, page_content='Tarjeta Éxito Digital Mastercard Ten control y manejo de tu Tarjeta Éxito Digital desde la APP Tuya. Haz tu primera compra presencial ingresando la tarjeta en el datáfono. Beneficios Ya no es posible solicitarla. Ahorra todo el año con los descuentos en los Almacene

#### Text spliter por oraciones completas

In [4]:
from langchain.text_splitter import NLTKTextSplitter

# Crear el splitter
nltk_splitter = NLTKTextSplitter()

# Dividir el texto
splits1_v3 = nltk_splitter.split_documents(docs1)

for split in splits1_v3:
    print(split)

page_content='¿Qué quieres buscar hoy?

Tus ofertas favoritas

Encuentra la tarjeta de crédito Tuya perfecta para ti y disfruta sus beneficios Con las tarjetas de crédito Tuya podrás comprar lo que más te gusta y disfrutar descuentos exclusivos en almacenes Éxito y Carulla.

Conócelas todas y solicita la tuya.

Conoce los beneficios de nuestras tarjetas: Disfruta descuentos exclusivos cada semana en almacenes Éxito y Carulla.

Con nuestras tarjetas podrás construir un buen historial crediticio.

Tarjeta Éxito Digital Mastercard Ten control y manejo de tu Tarjeta Éxito Digital desde la APP Tuya.

Haz tu primera compra presencial ingresando la tarjeta en el datáfono.

Beneficios Ya no es posible solicitarla.

Ahorra todo el año con los descuentos en los Almacenes Éxito y en tus marcas favoritas.

Usa tu Tarjeta desde la APP Tuya, para comprar por internet o para pagos sin contacto con Apple Pay (iOS), NFC (Android) y relojes Garmin y Fitbit.

Acumula Puntos Colombia en tus compras diferi

### 3. Generar Embeddings

all-MiniLM-L6-v2 - Balance ideal para español:

Dimensión: 384

Rendimiento: 58.7 en benchmark STS

Tamaño: 80MB

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings1 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},  # 'cuda' para GPU or 'cpu' 
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\PC\AppData\Local\Temp\ipykernel_10512\2426732458.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings1 = HuggingFaceEmbeddings(
C:\Users\PC\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 4. Almacenamiento Vectorial (FAISS)

Índice por defecto: IndexFlatL2 (búsqueda exacta)

(Facebook AI Similarity Search) es una biblioteca desarrollada por Facebook (Meta) para realizar búsquedas de similitud eficiente en vectores

In [6]:
from langchain.vectorstores import FAISS

vectorstore1 = FAISS.from_documents(
    documents=splits1_v3, # splits1
    embedding=embeddings1
)

# Guardar índice
vectorstore1.save_local("faiss_index_web_scraping")

# # Cargar desde disco
# loaded_store = FAISS.load_local(
#     "faiss_index_web_scraping",
#     embeddings1
# )

In [50]:
vectorstore1

#### splits1

In [51]:
# Recuperar documentos relevantes
docs = vectorstore1.similarity_search("tarjetas de crédito Tuya S.A.", k=3)
for doc in docs:
    print(doc.page_content)
    print("---")

recibirás nuevas ofertas pagando con las tarjetas de crédito Tuya. Así podrás ahorrar todo el año mientras compras todo lo que necesitas para el hogar y te das esos gusticos que siempre has querido. *Aplican términos y condiciones de cada Tarjeta disponible. Adquiere historial crediticio con nuestras tarjetas de crédito Tuya Para aquellas personas jóvenes o que aún no tienen historial crediticio, solicitar una de nuestras tarjetas de crédito es una excelente oportunidad para ir construyendo su historial de compras a crédito y así acceder más adelante a otros productos crediticios con mayor facilidad. Puedes empezar con alguna de nuestras tarjetas de crédito básicas como la y realizar compras a una sola cuota para no afectar tus finanzas personales. Si quieres conocer algunos consejos para el uso correcto de tu tarjeta de crédito, puedes consultar estos ¿Con la tarjeta de crédito puedo retirar dinero del cajero? Con las tarjetas de crédito Tuya puedes retirar dinero en todas las cajas
-

#### splits1_v3

In [7]:
# Recuperar documentos relevantes
docs = vectorstore1.similarity_search("tarjetas de crédito Tuya S.A.", k=3)
for doc in docs:
    print(doc.page_content)
    print("---")

Solicitar una tarjeta de crédito Tuya de forma presencial Si prefieres solicitar tu tarjeta de crédito de forma presencial, debes acercarte a uno de nuestros Centros de Atención ubicados en los principales Almacenes Éxito o Carulla del país.

Encuentra el más cercano Ten presente que debes presentar tu cédula original y nuestro asesor puede solicitar otros documentos de respaldo para el estudio de tu crédito.

Nota importante: la aprobación de la tarjeta de crédito está sujeta al análisis de riesgo realizado por Tuya S.A. Compañía de financiamiento.

¿Las tarjetas de crédito Tuya cuentan con algún seguro?

Según el tipo de tarjeta de crédito que tengas con nosotros, tendrás diferentes seguros y protecciones, puedes tener derecho desde un seguro de vida o un seguro de compra hasta garantías extendidas y asistencia global con Mastercard.

En cada una de nuestras tarjetas de crédito, en la sección Seguros, podrás conocer el detalle las coberturas de cada una.

Los seguros son ofrecidos po

### 5. Configuración de Llama 3.2 y Pipeline

#### device='cuda'

In [52]:
import torch

# Verifica la disponibilidad de CUDA
print(f"Dispositivo disponible: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "Ejecutando en CPU")

Dispositivo disponible: NVIDIA GeForce GTX 1660 Ti


In [2]:
import torch
print(f"Versión PyTorch: {torch.__version__}")
print(f"Versión CUDA: {torch.version.cuda}")
print(f"Capacidad de la GPU: {torch.cuda.get_device_capability()}")

Versión PyTorch: 2.1.0+cu118
Versión CUDA: 11.8
Capacidad de la GPU: (7, 5)


In [3]:
import torch
from flash_attn import flash_attn_qkvpacked_func

print(f"CUDA disponible: {torch.cuda.is_available()}")  # Debe ser True

ModuleNotFoundError: No module named 'flash_attn'

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from langchain.llms import HuggingFacePipeline
import torch

# ===== 1. Parámetros del Modelo =====
model_id = "unsloth/Llama-3.2-1B-Instruct"

# Configurar CUDA automáticamente
device = "cuda" if torch.cuda.is_available() else "cpu"
#device="cpu"

# ===== 2. Configuración de Quantización (solo para CPU) =====
quantization_config = None
if device == "cpu":
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

# ===== 3. Cargar Modelo y Tokenizer =====
try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        padding_side="left",
        trust_remote_code=True
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.bfloat16 if device == "cuda" else None,
        attn_implementation="flash_attention_2" if device == "cuda" else None
    )
except Exception as e:
    print(f"Error al cargar el modelo: {str(e)}")
    exit()

# ===== 4. Configuración del Pipeline =====
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.3,
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id,
    device=device  # Usar configuración automática
)

# ===== 5. Integración con LangChain =====
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "max_length": 2048,
        "truncation": True
    }
)

Error al cargar el modelo: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.


ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object.

: 

#### device='cpu'

In [8]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
from langchain.llms import HuggingFacePipeline
import torch

# ===== 1. Parámetros del Modelo =====
model_id1 = "unsloth/Llama-3.2-1B-Instruct"

# Configurar dispositivo
device1 = "cpu"  # Forzar CPU

# ===== 2. Cargar Modelo y Tokenizer =====
try:
    tokenizer1 = AutoTokenizer.from_pretrained(
        model_id1,
        padding_side="left",
        trust_remote_code=True
    )
    
    model1 = AutoModelForCausalLM.from_pretrained(
        model_id1,
        device_map=None,  # No usar device_map en CPU
        torch_dtype=torch.float32,  # Usar float32 en CPU
    )
except Exception as e:
    print(f"Error al cargar el modelo: {str(e)}")
    exit()

# ===== 3. Configuración del Pipeline =====
pipe1 = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
    max_new_tokens=512,  # Reducir para CPU
    temperature=0.3,
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,
    pad_token_id=tokenizer1.eos_token_id,
    device=device1  # Usar CPU
)

# ===== 4. Integración con LangChain =====
llm1 = HuggingFacePipeline(
    pipeline=pipe1,
    model_kwargs={
        "max_length": 1024,  # Reducir para CPU
        "truncation": True
    }
)

Device set to use cpu
C:\Users\PC\AppData\Local\Temp\ipykernel_10512\1410340790.py:47: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm1 = HuggingFacePipeline(


probar pipeline

In [9]:
# Ejemplo de uso
prompt = "¿Cuál es la tasa de interés de Credicompras?"
#response = llm(prompt)
response = llm1.invoke(prompt)
print(response)

?
La respuesta final es 6,5%.


In [10]:

# Ejemplo de uso
prompt = "¿ Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A?."
#response = llm(prompt)
response = llm1.invoke(prompt)
print(response)

 

Las tarjetas de crédito y débito más comunes incluyen:

1. Visa
2. Mastercard
3. American Express

También se pueden encontrar otras opciones como la tarjeta de pago en línea, pero estas no son tan comunes.

Si deseas obtener información adicional sobre cómo utilizar tus tarjetas o para realizar pagos, te recomiendo contactar con el servicio al cliente de tu banco o consultarte con un profesional financiero.


### 6. Configurar el Sistema RAG
Ahora, integra el vectorstore con el modelo para que pueda recuperar documentos relevantes antes de generar una respuesta:

In [11]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. Definir el prompt con contexto
template1 = """
<s>[INST] Eres un asistente especializado en productos financieros de Tuya S.A.
Responde la pregunta usando solo la información proporcionada en el contexto.

Contexto:
{context}

Pregunta: {question}
Respuesta: [/INST]
"""
prompt = PromptTemplate.from_template(template1)

# 2. Configurar la cadena de RAG
qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm1,  # Tu modelo Llama 3.2
    chain_type="stuff",
    retriever=vectorstore1.as_retriever(),  # Integrar el vectorstore
    chain_type_kwargs={"prompt": prompt}
)

In [14]:
question = "¿Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A.?"
response = qa_chain1.invoke({"query": question})
print(response["result"])

Efectivamente, Tuya S.A. ofrece varias opciones de tarjetas de crédito:

- **Carulla**: Una tarjeta de crédito diseñada específicamente para las mujeres.
- **Viva**: Una tarjeta de crédito que ofrece promociones y descuentos exclusivos en la aerolínea Viva.
- **Claro Pay Mastercard**: Una tarjeta de crédito que permite retiros en efectivo de las tarjetas Surtimayorista, almacenes Éxito, Carulla, Surtimax y en cajeros electrónicos en Colombia y en el exterior, con acceso a capital inmediato para llevarse lo que quieren con 0% de intereses en las compras que realice a una cuota.
- **Éxito Digital Mastercard**: Una tarjeta de crédito que permite comprar por internet y pagar por móviles.
- **Mastercard Gold Card**: Una tarjeta de crédito que ofrece beneficios exclusivos y promociones especiales.
- **Gold Mastercard**: Una tarjeta de crédito que ofrece beneficios exclusivos y promociones especiales.
- **Pro Mastercard**: Una tarjeta de crédito que ofrece beneficios exclusivos y promociones 

In [15]:
question = "¿Cuáles son los valores la tasa de interés y póliza del producto credicompras?."
response = qa_chain1.invoke({"query": question})
print(response["result"])

Pregunta: ¿Cuánto puede gastar con mi tarjeta de crédito?
Respuesta: No hay un límite máximo para el tipo de gasto que podremos realizar con tu tarjeta de crédito. Solo nos limita el importe de la compra.


## 2. usando document_loaders de langchain

### 1. Cargar Documentos desde URLs

In [1]:
from langchain.document_loaders import WebBaseLoader

urls = [
    "https://www.tuya.com.co/tarjetas-de-credito",
    "https://www.tuya.com.co/credicompras",
    "https://www.tuya.com.co/como-pago-mi-tarjeta-o-credicompras",
    "https://www.tuya.com.co/otras-soluciones-financieras",
    "https://www.tuya.com.co/nuestra-compania"
]

loader2 = WebBaseLoader(urls)
docs2 = loader2.load()



USER_AGENT environment variable not set, consider setting it to identify your requests.


### 2. Dividir Texto en Chunks

1000 tokens: Equivalente a ~1 página de texto, balance entre contexto y eficiencia

200 de overlap: Permite mantener contexto entre chunks adyacentes

In [2]:
docs2[0]

Document(metadata={'source': 'https://www.tuya.com.co/tarjetas-de-credito', 'title': 'Tarjetas de Crédito Tuya: para comprar lo que más te gusta', 'description': 'Con las tarjetas de crédito Tuya podrás comprar lo que quieras y disfrutar de descuentos exclusivos en Éxito y Carulla. Solicítala aquí.', 'language': 'es'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTarjetas de Crédito Tuya: para comprar lo que más te gusta\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Pasar al contenido principal\n    \n\n\n\n\n\n\n\n\n\n\nYo tengo\n\n\n\n\n\n\n\n\n\n\nSoluciones de pago\n\nTransacciones\n\nQuiero pagar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMain navigation\n\n\nInicio\n\n\nDisfruta \n\n\nOfertas\n\n\nApp Tuya\n\n\n\n\nDónde pagas\n\n\nProductos y servicios \n\n\nTarjeta de crédito\n\n\nTarjeta Éxito\n\n\nTarjeta Éxito Mastercard\n\n\nTarjeta Carulla Gold Mastercard \n\n\nTarjeta Carulla Mastercard Black\n\

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
#from utils import clean_text
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios múltiples
    return text.strip()

# 3. Preprocesar los documentos
def preprocess_documents(docs):
    cleaned_docs = []
    for doc in docs:
        cleaned_content = clean_text(doc.page_content)
        cleaned_docs.append(Document(page_content=cleaned_content, metadata=doc.metadata))
    return cleaned_docs

# 4. Aplicar preprocesamiento
cleaned_docs = preprocess_documents(docs2)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100 #200
)

splits2 = text_splitter.split_documents(cleaned_docs)

#### Text splitter por caracteres 

In [31]:
splits2

[Document(metadata={'source': 'https://www.tuya.com.co/tarjetas-de-credito', 'title': 'Tarjetas de Crédito Tuya: para comprar lo que más te gusta', 'description': 'Con las tarjetas de crédito Tuya podrás comprar lo que quieras y disfrutar de descuentos exclusivos en Éxito y Carulla. Solicítala aquí.', 'language': 'es'}, page_content='Tarjetas de Crédito Tuya: para comprar lo que más te gusta Pasar al contenido principal Yo tengo Soluciones de pago Transacciones Quiero pagar Main navigation Inicio Disfruta Ofertas App Tuya Dónde pagas Productos y servicios Tarjeta de crédito Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold Mastercard Tarjeta Carulla Mastercard Black Tarjeta Viva Mastercard Tarjeta Club del Comerciante Mastercard Tarjeta Claro Pay Cuenta Digital TuyaPay CrediCompras Seguros y asistencias Otras soluciones financieras Conócenos Nuestra compañía Gobierno corporativo Sostenibilidad Atención a inversionistas Soluciones de Pago Activar Tarjeta Trabaja con nosotros M

#### Text splitter por token

In [5]:
from langchain.text_splitter import TokenTextSplitter

# Crear el splitter
token_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=20)

# Dividir el texto
splits2_v2 = token_splitter.split_documents(cleaned_docs)

for split in splits2_v2:
    print(split)

page_content='Tarjetas de Crédito Tuya: para comprar lo que más te gusta Pasar al contenido principal Yo tengo Soluciones de pago Transacciones Quiero pagar Main navigation Inicio Disfruta Ofertas App Tuya Dónde pagas Productos y servicios Tarjeta de crédito Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold' metadata={'source': 'https://www.tuya.com.co/tarjetas-de-credito', 'title': 'Tarjetas de Crédito Tuya: para comprar lo que más te gusta', 'description': 'Con las tarjetas de crédito Tuya podrás comprar lo que quieras y disfrutar de descuentos exclusivos en Éxito y Carulla. Solicítala aquí.', 'language': 'es'}
page_content=' Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold Mastercard Tarjeta Carulla Mastercard Black Tarjeta Viva Mastercard Tarjeta Club del Comerciante Mastercard Tarjeta Claro Pay Cuenta Digital TuyaPay CrediCompras Seguros y asistencias Otras soluciones financieras Conócenos Nuestra compañía Gobierno' metadata={'source': 'https://www.tuya.com.co

#### Text spliter por oraciones completas

In [ ]:
from langchain.text_splitter import NLTKTextSplitter

# Crear el splitter
nltk_splitter = NLTKTextSplitter()

# Dividir el texto
splits2_v3 = nltk_splitter.split_documents(cleaned_docs)

for split in splits2_v3:
    print(split)

page_content='Tarjetas de Crédito Tuya: para comprar lo que más te gusta Pasar al contenido principal Yo tengo Soluciones de pago Transacciones Quiero pagar Main navigation Inicio Disfruta Ofertas App Tuya Dónde pagas Productos y servicios Tarjeta de crédito Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold Mastercard Tarjeta Carulla Mastercard Black Tarjeta Viva Mastercard Tarjeta Club del Comerciante Mastercard Tarjeta Claro Pay Cuenta Digital TuyaPay CrediCompras Seguros y asistencias Otras soluciones financieras Conócenos Nuestra compañía Gobierno corporativo Sostenibilidad Atención a inversionistas Soluciones de Pago Activar Tarjeta Trabaja con nosotros Menú Llámanos Preguntas Frecuentes Buscar ¿Dónde consultar?

Llámanos Visítanos ¿Qué quieres buscar hoy?

You must have JavaScript enabled to use this form.

Buscar Quizá estés buscando: Usos y beneficios Extracto Pérdida o hurto, solicitudes de quejas y cancelaciones Realización de pagos Avances y cupo Solicitud y activa

### 3. Generar Embeddings

all-MiniLM-L6-v2 - Balance ideal para español:

modelo compacto pero potente entrenado para generar incrustaciones de oraciones de alta calidad . Toma una oración o un párrafo corto y lo asigna a un vector de 384 dimensiones, lo que facilita su uso para tareas como: Búsqueda semántica (encontrar oraciones similares) Agrupamiento (agrupar oraciones por significado)

Dimensión: 384

Rendimiento: 58.7 en benchmark STS

Tamaño: 80MB

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings2 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},  # 'cuda' para GPU
    encode_kwargs={'normalize_embeddings': True}
)

### 4. Almacenamiento Vectorial (FAISS)

Índice por defecto: IndexFlatL2 (búsqueda exacta)

(Facebook AI Similarity Search) es una biblioteca desarrollada por Facebook (Meta) para realizar búsquedas de similitud eficiente en vectores

In [33]:
from langchain.vectorstores import FAISS

vectorstore2 = FAISS.from_documents(
    documents=splits2_v3, #splits2_v3 , splits2_v2, splits2_v3
    embedding=embeddings2
)

# Guardar índice
vectorstore2.save_local("faiss_index_langchain")

# # Cargar desde disco
# loaded_store = FAISS.load_local(
#     "faiss_index_langchain",
#     embeddings1
# )

#### splits2

In [30]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjetas de crédito Tuya S.A.", k=5)
for doc in docs:
    print(doc.page_content)
    print("---")

Para aquellas personas jóvenes o que aún no tienen historial crediticio, solicitar una de nuestras tarjetas de crédito es una excelente oportunidad para ir construyendo su historial de compras a crédito y así acceder más adelante a otros productos crediticios con mayor facilidad. Puedes empezar con alguna de nuestras tarjetas de crédito básicas como la Tarjeta Éxito y realizar compras a una sola cuota para no afectar tus finanzas personales. Si quieres conocer algunos consejos para el uso correcto de tu tarjeta de crédito, puedes consultar estos tips de educación financiera. ¿Con la tarjeta de crédito puedo retirar dinero del cajero? Con las tarjetas de crédito Tuya puedes retirar dinero en todas las cajas de los almacenes del Grupo Éxito y con las tarjetas Éxito Gold Mastercard y la Éxito Pro Mastercard también puedes retirar dinero en los cajeros automáticos. ¿Con las tarjetas de crédito Tuya puedo hacer compras fuera de Colombia? Si tienes una de nuestras tarjetas de crédito Éxito
-

In [31]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjeta Éxito Pro Mastercard", k=3)
for doc in docs:
    print(doc.page_content)
    print("---")

¡Solicítala aquí! Tarjeta Éxito Esta tarjeta dejó de funcionar a partir del 2023 y no es posible solicitarla. Este contenido es informativo. Conoce los productos que tenemos disponibles en nuestros Centros de Atención Tarjeta Éxito. Beneficios Paga la cuota de manejo solo cuando la uses o cuando tengas una deuda pendiente. Realiza el pago de tu Tarjeta Éxito Tuya y saca plata en las cajas de los almacenes del Grupo Éxito. Ahorra todo el año con los descuentos en los Almacenes Éxito. Difiere tus compras de 1 a 48 meses. ¡Quiero conocer más! Tarjeta Éxito Pro Mastercard Disfruta de los beneficios exclusivos que tenemos para ti con nuestra Tarjeta Éxito Gold Mastercard. Esta tarjeta no se encuentra disponible desde el 2022. Conoce los productos que tenemos disponibles en nuestros Centros de Atención Tarjeta Éxito. Beneficios Haz tu primera compra presencial ingresando la tarjeta en el datáfono. Paga la cuota de manejo solo cuando la uses o cuando tengas una deuda pendiente. Úsala donde
--

#### splits2_v2

In [8]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjetas de crédito Tuya S.A.", k=5)
for doc in docs:
    print(doc.page_content)
    print("---")

ón de la tarjeta de crédito está sujeta al análisis de riesgo realizado por Tuya S.A. Compañía de financiamiento. ¿Las tarjetas de crédito Tuya cuentan con algún seguro? Según el tipo de tarjeta de crédito que tengas con nosotros, tendrás dif
---
 recibirás nuevas ofertas pagando con las tarjetas de crédito Tuya. Así podrás ahorrar todo el año mientras compras todo lo que necesitas para el hogar y te das esos gusticos que siempre has querido. *Aplican términos y condiciones de cada Tarjeta disponible. Adquiere historial crediticio
---
 de crédito Encuentra la tarjeta de crédito Tuya perfecta para ti y disfruta sus beneficios Con las tarjetas de crédito Tuya podrás comprar lo que más te gusta y disfrutar descuentos exclusivos en almacenes Éxito y Carulla. Conócelas todas y solicita la tuya. Conoce los beneficios de nu
---
ución de los mismos. ¿Cómo pagar las tarjetas de crédito Tuya? Cuando tengas una de las tarjetas de crédito Tuya, podrás realizar todos tus pagos de diferentes maneras

In [9]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjeta Éxito Pro Mastercard", k=3)
for doc in docs:
    print(doc.page_content)
    print("---")

 Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold Mastercard Tarjeta Carulla Mastercard Black Tarjeta Viva Mastercard Tarjeta Club del Comerciante Mastercard Tarjeta Claro Pay Cuenta Digital TuyaPay CrediCompras Seguros y asistencias Otras soluciones financieras Conócenos Nuestra compañía Gobierno
---
o Tarjeta Éxito Tarjeta Éxito Mastercard Tarjeta Carulla Gold Mastercard Tarjeta Carulla Mastercard Black Tarjeta Viva Mastercard Tarjeta Club del Comerciante Mastercard Tarjeta Claro Pay Cuenta Digital TuyaPay CrediCompras Seguros y asistencias Otras soluciones financieras Conócenos Nuestra compañía Gobier
---
eta Éxito Mastercard Vive tu primera experiencia crediticia con la Tarjeta Éxito Mastercard, una tarjeta de crédito pensada para que puedas disfrutar nuevas posibilidades. Haz tu primera compra presencial ingresando la tarjeta en el datáfono. Beneficios Ahorra todo el año con descuentos en Almacenes Éx
---


#### splits2_v3

In [34]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjetas de crédito Tuya S.A.", k=5)
for doc in docs:
    print(doc.page_content)
    print("---")

Puedes realizar el pago de tu tarjeta de crédito 24/7 y en cualquiera de las dos plataformas habilitadas para ello.

Conoce cuales: Botón de pagos Bancolombia Botón de pagos PSE Paga tu tarjeta de crédito de forma presencial Tenemos diferentes aliados para que puedas realizar el pago de tu tarjeta de crédito de forma presencial.

Escoge la opción que más cerca te quede y conoce cómo pagar tu tarjeta de crédito en corresponsales Efecty Moviired Cajas de pago del Grupo Éxito Bancos autorizados Dónde puedo comprar con las tarjetas de crédito Tuya Puedes comprar en nuestros comercios aliados con la Tarjeta Éxito y en cualquier establecimiento comercial a nivel nacional con las tarjetas Éxito Gold Mastercard, Éxito Pro Mastercard, Carulla Gold Mastercard y Carulla Mastercard Black.

Acumula Puntos Colombia en las compras que realices con las tarjetas de crédito Tuya Con nuestras tarjetas de crédito puedes realizar compras en diferentes establecimientos comerciales, acumular Puntos Colombia 

In [35]:
# Recuperar documentos relevantes
docs = vectorstore2.similarity_search("tarjeta Éxito Pro Mastercard", k=3)
for doc in docs:
    print(doc.page_content)
    print("---")

Conoce los productos que tenemos disponibles en nuestros Centros de Atención Tarjeta Éxito.

Beneficios Paga la cuota de manejo solo cuando la uses o cuando tengas una deuda pendiente.

Realiza el pago de tu Tarjeta Éxito Tuya y saca plata en las cajas de los almacenes del Grupo Éxito.

Ahorra todo el año con los descuentos en los Almacenes Éxito.

Difiere tus compras de 1 a 48 meses.

¡Quiero conocer más!

Tarjeta Éxito Pro Mastercard Disfruta de los beneficios exclusivos que tenemos para ti con nuestra Tarjeta Éxito Gold Mastercard.

Esta tarjeta no se encuentra disponible desde el 2022.

Conoce los productos que tenemos disponibles en nuestros Centros de Atención Tarjeta Éxito.

Beneficios Haz tu primera compra presencial ingresando la tarjeta en el datáfono.

Paga la cuota de manejo solo cuando la uses o cuando tengas una deuda pendiente.

Úsala donde quieras, en compras nacionales, internacionales o por internet.

Saca plata en todas las cajas de los almacenes del Grupo Éxito y en

### 5. Configuración de Llama 3.2 y Pipeline

Usar device='cpu'

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
from langchain.llms import HuggingFacePipeline
import torch

# ===== 1. Parámetros del Modelo =====
model_id2 = "unsloth/Llama-3.2-1B-Instruct"

# Configurar dispositivo
device2 = "cpu"  # Forzar CPU

# ===== 2. Cargar Modelo y Tokenizer =====
try:
    tokenizer2 = AutoTokenizer.from_pretrained(
        model_id2,
        padding_side="left",
        trust_remote_code=True
    )
    
    model2 = AutoModelForCausalLM.from_pretrained(
        model_id2,
        device_map=None,  # No usar device_map en CPU
        torch_dtype=torch.float32,  # Usar float32 en CPU
    )
except Exception as e:
    print(f"Error al cargar el modelo: {str(e)}")
    exit()

# ===== 3. Configuración del Pipeline =====
pipe2 = pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
    max_new_tokens=512,  # Reducir para CPU
    temperature=0.3, # controla la aleatoriedad de las respuestas. Un valor más bajo (cercano a 0) hace que el modelo sea más determinista
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,
    pad_token_id=tokenizer2.eos_token_id,
    device=device2  # Usar CPU
)

# ===== 4. Integración con LangChain =====
llm2 = HuggingFacePipeline(
    pipeline=pipe2,
    model_kwargs={
        "max_length": 1024,  # Reducir para CPU
        "truncation": True
    }
)

Device set to use cpu


probar pipeline

In [11]:

# Ejemplo de uso
prompt = "¿Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A?."
#response = llm(prompt)
response = llm2.invoke(prompt)
print(response)

?

Para esta pregunta, necesitaré obtener información a través de internet o contactar directamente con la empresa. 

Hasta donde llega mi conocimiento (abril de 2023), Tuya S.A., una empresa china especializada en tecnología de inteligencia artificial y IoT, ofrece varias opciones para sus clientes. Aunque no tengo acceso directo a su sitio web actualizado, puedo proporcionarte información general sobre algunas de las tarjetas que han sido ofrecidas por el negocio.

Tuya S.A. ha lanzado varios productos relacionados con la conectividad Wi-Fi y la automatización de dispositivos, incluyendo:

1. **Tarjetas Wi-Fi**: Estas tarjetas permiten conectar dispositivos electrónicos a Internet de las cosas (IoT) sin necesidad de cables. Ofrecen una conexión segura y rápida a Internet.
2. **Dispositivos IoT**: Algunos de estos dispositivos pueden ser controlados remotamente mediante aplicaciones móviles, lo cual puede ser útil para automotivistas, agricultores, etc.
3. **Sistemas de seguridad avan

En este punto el modelo no tiene ningun tipo de contexto

### 6. Configurar el Sistema RAG
Ahora, integra el vectorstore con el modelo para que pueda recuperar documentos relevantes antes de generar una respuesta:

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. Definir el prompt con contexto
template1 = """
<s>[INST] Eres un asistente especializado en productos financieros de Tuya S.A.
Responde la pregunta usando solo la información proporcionada en el contexto.

Contexto:
{context}

Pregunta: {question}
Respuesta: [/INST]
"""
prompt = PromptTemplate.from_template(template1)

# 2. Configurar la cadena de RAG
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm2,  # Tu modelo Llama 3.2
    chain_type="stuff",
    retriever=vectorstore2.as_retriever(),  # Integrar el vectorstore # search_kwargs={"k": 3} para limitar fragmentos
    chain_type_kwargs={"prompt": prompt}
)

#### splits2

In [39]:
question = "¿Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A.?"
response = qa_chain2.invoke({"query": question})
print(response["result"])

Tarjeta Éxito Digital Mastercard Ten control y manejo de tu tarjeta desde la App Tuya. Haz tu primera compra presencial ingresando la tarjeta en el datáfono. Beneficios Ya no es posible solicitarla. Ahorra todo el año con los descuentos en los Almacenes Éxito y en tus marcas favoritas. Usualmente, las tarjetas de crédito de Tuya S.A. están disponibles en dos tipos:

1. Tarjeta Éxito Digital Mastercard: Es una tarjeta digital que permite el acceso remoto a la plataforma Tuya, permitiendo acceder a servicios y aplicaciones relacionadas con las finanzas personales y empresariales.
2. Tarjeta Éxito: Es una tarjeta tradicional que ofrece beneficios adicionales como descuentos en los Almacenes Éxito y otras ventajas específicas para los clientes de Tuya S.A.

Es importante mencionar que la disponibilidad de estas tarjetas puede variar dependiendo del país y la región en la cual opera Tuya S.A.


In [40]:
question = "¿Cuáles son los valores la tasa de interés y póliza del producto Credicompras?."
response = qa_chain2.invoke({"query": question})
print(response["result"])

Tasa de interés: 0% (término variable) Política de pólizas: 6 meses Plazo mínimo: 12 meses Plazo máximo: 36 meses Monto inicial: $1.000.000 - $15.000.000. La política de pólizas incluye dos opciones:

*   Política de pólizas básica: 3 años de plazo mínimo de 12 meses y 4 años de plazo máximo de 36 meses.
*   Política de pólizas avanzada: 5 años de plazo mínimo de 24 meses y 10 años de plazo máximo de 60 meses.


#### splits2_v2

In [17]:
question = "¿Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A.?"
response = qa_chain2.invoke({"query": question})
print(response["result"])

Tarjeta Éxito Digital Mastercard; Tarjeta Éxito Digital; Tarjeta Éxito Visa; Tarjeta Éxito American Express; Tarjeta Éxito Discover; Tarjeta Éxito Diners Club; Tarjeta Éxito JCB; Tarjeta Éxito Maestro; Tarjeta Éxito Debito; Tarjeta Éxito Credito; Tarjeta Éxito Plano de Pagos; Tarjeta Éxito Plus; Tarjeta Éxito Premium; Tarjeta Éxito Gold; Tarjeta Éxito Silver; Tarjeta Éxito Platinum; Tarjeta Éxito Diamond; Tarjeta Éxito Elite; Tarjeta Éxito Premier; Tarjeta Éxito Business; Tarjeta Éxito Executive; Tarjeta Éxito VIP; Tarjeta Éxito Especial; Tarjeta Éxito Extra; Tarjeta Éxito Exclusiva; Tarjeta Éxito Aventura; Tarjeta Éxito Amigo; Tarjeta Éxito Familia; Tarjeta Éxito Ciudadanos; Tarjeta Éxito Nacionales; Tarjeta Éxito Nacional Banco; Tarjeta Éxito Banco Santander; Tarjeta Éxito Banco BBVA; Tarjeta Éxito Banco HSBC; Tarjeta Éxito Banco BCP; Tarjeta Éxito Banco CaixaBank; Tarjeta Éxito Banco Bankia; Tarjeta Éxito Banco Sabadell; Tarjeta Éxito Banco Unicaja; Tarjeta Éxito Banco Bankinter; Ta

In [15]:
question = "¿Cuáles son los valores la tasa de interés y póliza del producto Credicompras?."
response = qa_chain2.invoke({"query": question})
print(response["result"])

La respuesta completa sería:

*   **Tasa de Interés:** La tasa de interés de la póliza de seguro de deuda es del 8% anual sin intereses ni cargos adicionales. Esta tasa puede variar según la condición financiera del titular de la póliza y otros factores relevantes.
*   **Política de Pólizas:** Tuya S.A. ofrece diferentes políticas de pólizas, incluyendo la Política de Credicompras. Esta política establece los términos y condiciones bajo los cuales se otorga el seguro de deudas. Algunos aspectos clave de esta política son:
    +   El titular de la póliza debe tener una buena reputación crediticia y estar entre los 18 y 65 años de edad.
    +   No se pueden obtener polizas con matices de pago o retrasos en pagos.
    +   Se requiere un depósito inicial mínimo de COP 100.000.000 (Cotización mínima).
    +   Las pólizas están sujetas a ciertas limitaciones e incompatibilidades con otras formas de garantías o contratos similares.
    +   Es posible cancelar la póliza si el titular ha alcanz

#### splits2_v3

In [42]:
question = "¿Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A.?"
response = qa_chain2.invoke({"query": question})
print(response["result"])

[Tuya S.A.] ofrece varias opciones de tarjetas de crédito diseñadas para satisfacer las necesidades de diversas industrias y sectores. Entre ellas, hay algunas de las siguientes:

-   Tarjeta Éxito

-   Tarjeta Carulla

-   Tarjeta Viva

-   Tarjeta Club del Comerciante

-   Tarjeta de Crédito Claro Pay Mastercard

Es importante destacar que estas tarjetas están diseñadas para ofrecer beneficios especiales a ciertos grupos de clientes, pero también pueden beneficiarse de ellas cualquier persona que busque obtener acceso a servicios financieros asequibles y convenientes.


In [39]:
question = "¿Cuáles son los valores la tasa de interés y póliza del producto Credicompras?."
response = qa_chain2.invoke({"query": question})
print(response["result"])

Título: Credicompras

Descripción:

CrediCompras es una opción de crédito personal diseñada para ayudarte a alcanzar tus objetivos financieros personales. Ofrece una variedad de opciones de crédito con tasas de interés competitivas y plazos flexibles, todo ello en un proceso sencillo y transparente.

Beneficios clave:

- Plazos flexibles: puedes elegir entre plazos de 12, 18, 24 y 36 meses.
- Tasas de interés competitivas: obtenemos tasas de interés anuales muy bajas, especialmente para los primeros años de la póliza.
- Opciones de crédito personalizadas: puedes elegir entre diferentes tipos de crédito, como crédito personal, crédito hipotecario, etc.
- Acceso a productos de la cadena Éxito: puedes comprar productos de alta calidad en los almacenes Éxito, siempre que cumpla con ciertos requisitos.
- Servicio rápido y eficiente: podemos ayudarte a resolver cualquier problema que surja durante el proceso de crédito.

Estructura del producto:

- Póliza colectiva Seguro deudores: ofrece un

In [41]:
question = "¿Cuáles son los valores la tasa de interés y póliza del producto Credicompras?."
response = qa_chain2.invoke({"query": question})
print(response["result"])

Título: Póliza colectiva seguro deudores

Fecha: [Fecha]

Objetivo:

El objetivo principal de esta póliza colectiva seguro deudores es brindar a los clientes de Tuya S.A. una opción de seguro que les permita proteger su crédito contra posibles pérdidas debido a eventos imprevistos como incumplimiento de pagos, insolvencia de terceros, etc.

Descripción general:

Esta póliza colectiva seguro deudores ofrece una cobertura de $3.159 por millón desembolsado, que comprende dos tipos de coberturas: 

- Cobertura de garantía: Esta cobertura cubre el valor de la deuda después de haber sido adeudado por el titular del crédito.
- Cobertura de responsabilidad civil: Esta cobertura cubre los costos legales y médicos derivados de cualquier acción judicial que pueda ser impuesta contra el titular del crédito por cualquier evento imprevisto.

Beneficios adicionales:

- La cobertura de garantía cubre hasta 90% del valor de la deuda después de haber sido adeudado por el titular del crédito.
- La cobert

##### sistema que genere 5 preguntas adicionales para mejorar la recuperación de información en un pipeline RAG con Llama 3.2,

1. Template de Prompt para Llama 3.2
El prompt debe guiar al modelo para generar variantes útiles de la pregunta original. Ejemplo:

In [ ]:
from langchain.prompts import PromptTemplate

query_expansion_template = """
<|begin_of_text|>
<s>[INST] Eres un asistente de expansión de consultas. Genera 5 variantes de la siguiente pregunta para ayudar a recuperar documentos relevantes de la base vectorial. 
Sigue estas reglas:
1. Mantén el significado original.
2. Usa sinónimos y reestructuraciones gramaticales.
3. Incluye aspectos específicos implícitos en la pregunta.
4. Formato: Lista numerada.

Pregunta Original: {question}

Variantes Generadas:
[/INST]
"""
prompt = PromptTemplate.from_template(query_expansion_template)

2. Generación de Preguntas con Llama 3.2
Integra el template en el pipeline de generación:

In [42]:
from langchain.chains import LLMChain

# Configurar cadena de expansión
query_expansion_chain = LLMChain(
    llm=llm2,  # Tu instancia de Llama 3.2
    prompt=prompt,
    output_key="expanded_queries"
)

# Función para generar variantes
def generate_query_variations(question):
    response = query_expansion_chain.invoke({"question": question})
    queries = [question]  # Incluir la original
    # Parsear la respuesta (ej: "1. ...\n2. ...")
    for line in response["expanded_queries"].split("\n"):
        if line.strip():
            queries.append(line.split(". ", 1)[1].strip())
    return queries[:6]  # 5 variantes + original

C:\Users\PC\AppData\Local\Temp\ipykernel_19556\294546713.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_expansion_chain = LLMChain(


3. Integración con RAG
Modifica el pipeline de recuperación para usar todas las preguntas:

In [43]:
from langchain.retrievers import EnsembleRetriever

def enhanced_retrieval(question, vectorstore):
    # Paso 1: Generar variantes
    all_queries = generate_query_variations(question)
    
    # Paso 2: Recuperar documentos para cada consulta
    retrievers = []
    for query in all_queries:
        retriever = vectorstore.as_retriever(
            search_type="mmr",  # Maximal Marginal Relevance
            search_kwargs={"k": 3}
        )
        retrievers.append(retriever)
    
    # Paso 3: Combinar resultados con Ensemble Retriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=retrievers,
        weights=[1.0/len(retrievers)]*len(retrievers)  # Peso uniforme
    )
    
    return ensemble_retriever.get_relevant_documents(question)

4. Pipeline Completo (RAG + Expansión)

In [44]:
from langchain.chains import RetrievalQA

# Configurar cadena final
qa_chain = RetrievalQA.from_chain_type(
    llm=llm2,
    chain_type="stuff",
    retriever=enhanced_retrieval,  # Nuestra función personalizada
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template="Responde basado en este contexto:\n{context}\nPregunta: {question}",
            input_variables=["context", "question"]
        )
    }
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Input should be a valid dictionary or instance of BaseRetriever [type=model_type, input_value=<function enhanced_retrie...l at 0x0000023E48F4BEC0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [ ]:
question = "¿Qué tarjetas de crédito ofrece Tuya S.A.?"
expanded_queries = generate_query_variations(question)
print("Variantes generadas:")
for q in expanded_queries:
    print(f"- {q}")

# Resultado esperado:
# - ¿Cuáles son las tarjetas de crédito disponibles en Tuya S.A.?
# - ¿Qué opciones de tarjetas de crédito tiene Tuya?
# - ¿Puede listar las tarjetas de crédito ofrecidas por Tuya?
# - ¿Qué productos crediticios de tarjeta ofrece Tuya S.A.?
# - ¿Cuáles son las tarjetas de crédito que puedo solicitar en Tuya?

docs = enhanced_retrieval(question, vectorstore)
answer = qa_chain.invoke({"query": question})
print(answer["result"])

In [ ]:
# Versión profesional con logging y validación
import logging
from tqdm.auto import tqdm

logger = logging.getLogger(__name__)

def build_rag_pipeline(docs):
    try:
        logger.info("Iniciando carga de documentos...")
        # loader = WebBaseLoader(urls)
        # docs = loader.load()
        
        logger.info(f"Documentos cargados: {len(docs)}")
        
        logger.info("Procesando chunks...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=lambda x: len(x.split())  # Contar por palabras
        )
        splits = text_splitter.split_documents(docs)
        
        logger.info(f"Chunks generados: {len(splits)}")
        
        logger.info("Generando embeddings...")
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'},  # 'cuda' para GPU
            encode_kwargs={'normalize_embeddings': True}
        )
        
        logger.info("Construyendo vector store...")
        vectorstore = FAISS.from_documents(
            splits,
            embeddings
        )
        
        return vectorstore
    
    except Exception as e:
        logger.error(f"Error en el pipeline: {str(e)}")
        raise